In [1]:
import sys
sys.path.append("..")
import numpy as np
from project.ship import Ship
from project.tests import Tests
from project.botma import TMA
import time
from pathlib import Path
from datetime import datetime

In [2]:
# Класс для сохранения результатов
tests = Tests('test1')
Path("../tests/test1/d30").mkdir(parents=True, exist_ok=True)
print("Время последнего моделирования:", datetime.now().strftime("%d/%m/%Y %H:%M"))

# Создаем наблюдателя
observer_x, observer_y, observer_course, observer_velocity = 0, 0, 0, 5
observer = Ship('Наблюдатель', observer_x, observer_y, observer_course,
                observer_velocity, verbose=True)

Время последнего моделирования: 22/03/2021 21:52


In [3]:
# Моделирование траектории наблюдателя
observer.forward_movement(3 * 60)
observer.change_course(270, 'left', omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, 'right', omega=0.5)
observer.forward_movement(3 * 60)

Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с


In [4]:
def target_func(seed=None):
    np.random.seed(seed)
    b = 0
    d = 30
    c = np.random.uniform(0, 180)
    v = np.random.uniform(3, 15)
    return [b, d, c, v]
p0 = [0., 25., 90., 7.]
std_arr = [0., 0.1, 0.2, 0.3, 0.5, 1.]
precisions_1 = ['{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}',
'{0:.2f}','{0:.3f}', '{0:.2f}','{0:.3f}','{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{:.0f}', '{0:.4f}', '{0:.0f}', '{0:.0f}']
precisions_2 = ['{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.2f}','{0:.3f}', '{0:.2f}','{0:.3f}', '{0:.2f}', '{0:.2f}', '{0:.0f}', '{0:.4f}']

In [5]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2)
    dict_results = tma.swarm(n=100, target_func=target_func, p0=p0, seeded=True, verbose=True)
    df = tests.get_df(dict_results)
    tma.print_verbose()

    print('Показатели Монте-Карло:')
    for i in range(4):
        a = df['Точ'].apply(lambda x: x[i])
        print('Вариант {}: Рэф = {:.0f}%, Рэф_мин = {:.0f}'.format(i, 100 * a.mean(), min(a)))
    print('Ка = {:.2f}'.format(df['Ка'].mean()))
    print('Nf = {:.0f}, Nf_max = {:.0f}'.format(df['Nf'].mean(), max(df['Nf'])))
    print('Iter = {:.0f}, Iter_max = {:.0f}'.format(df['Iter'].mean(), max(df['Iter'])))
    print('t = {:.4f}, t_max = {:.4f}'.format(df['t'].mean(), max(df['t'])))

    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['СКО X', 'СКО Y', 'СКО VX', 'СКО VY', 'Кб', 'П0_апр', 'Д0_апр', 'К0_апр', 'V0_апр'], axis=1).head())

    tests.save_df(df, name='../tests/test1/d30/ЛМ, ' + str(np.degrees(tma.standart_deviation)) + '.xlsx')
    print('_________________________________________________')

Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 1.0 с
СКОп = 0.0, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.00
Nf = 9, Nf_max = 11
Iter = 8, Iter_max = 10
t = 0.0035, t_max = 0.0116


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t,Nf,Iter
0,0.0,30.000,98.786,11.58,0.0,30.000,98.787,11.58,30.54,29.331,30.54,29.330,0.00,"[1, 1, 1, 1]",0.0083,10,9
1,0.0,30.000,75.064,11.64,0.0,30.000,75.064,11.64,25.34,34.283,25.34,34.283,0.00,"[1, 1, 1, 1]",0.0067,10,9
2,0.0,30.000,78.479,3.31,360.0,30.000,78.479,3.31,10.19,28.629,10.19,28.629,0.00,"[1, 1, 1, 1]",0.0056,8,7
3,0.0,30.000,99.144,11.50,0.0,30.000,99.144,11.50,30.42,29.213,30.42,29.213,0.00,"[1, 1, 1, 1]",0.0066,10,9
4,0.0,30.000,174.065,9.57,360.0,30.000,174.065,9.57,8.40,16.138,8.40,16.138,0.00,"[1, 1, 1, 1]",0.0046,8,7


_________________________________________________
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.8 с
СКОп = 0.1, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 98%, Рэф_мин = 0
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.01
Nf = 9, Nf_max = 11
Iter = 8, Iter_max = 10
t = 0.0026, t_max = 0.0055


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t,Nf,Iter
0,0.0,30.000,98.786,11.58,360.0,30.115,98.530,11.68,30.54,29.331,30.57,29.523,0.01,"[1, 1, 1, 1]",0.0031,10,9
1,0.0,30.000,75.064,11.64,0.0,29.819,75.130,11.53,25.34,34.283,25.32,34.022,0.01,"[1, 1, 1, 1]",0.0026,10,9
2,0.0,30.000,78.479,3.31,0.0,30.116,81.405,3.27,10.19,28.629,10.21,28.537,0.01,"[1, 1, 1, 1]",0.0023,8,7
3,0.0,30.000,99.144,11.50,0.0,29.808,99.551,11.37,30.42,29.213,30.41,28.905,0.01,"[1, 1, 1, 1]",0.0027,10,9
4,0.0,30.000,174.065,9.57,0.0,30.020,174.209,9.65,8.40,16.138,8.38,16.049,0.01,"[1, 1, 1, 1]",0.0023,8,7


_________________________________________________
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.8 с
СКОп = 0.2, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 91%, Рэф_мин = 0
Вариант 1: Рэф = 98%, Рэф_мин = 0
Вариант 2: Рэф = 98%, Рэф_мин = 0
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.04
Nf = 9, Nf_max = 11
Iter = 8, Iter_max = 10
t = 0.0026, t_max = 0.0045


...

_________________________________________________
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.8 с
СКОп = 0.5, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 51%, Рэф_мин = 0
Вариант 1: Рэф = 88%, Рэф_мин = 0
Вариант 2: Рэф = 88%, Рэф_мин = 0
Вариант 3: Рэф = 92%, Рэф_мин = 0
Ка = 0.25
Nf = 9, Nf_max = 11
Iter = 8, Iter_max = 10
t = 0.0027, t_max = 0.0062


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t,Nf,Iter
0,0.0,30.000,98.786,11.58,359.9,30.585,97.534,12.06,30.54,29.331,30.66,30.309,0.25,"[1, 1, 1, 1]",0.0057,10,9
1,0.0,30.000,75.064,11.64,0.0,29.122,75.397,11.11,25.34,34.283,25.24,33.019,0.25,"[1, 1, 1, 1]",0.0027,10,9
2,0.0,30.000,78.479,3.31,0.1,30.577,93.723,3.19,10.19,28.629,10.27,28.164,0.26,"[0, 0, 0, 0]",0.0028,8,7
3,0.0,30.000,99.144,11.50,0.1,29.068,101.216,10.86,30.42,29.213,30.37,27.720,0.26,"[0, 1, 1, 1]",0.0046,10,9
4,0.0,30.000,174.065,9.57,0.0,30.102,174.754,10.00,8.40,16.138,8.31,15.697,0.24,"[1, 1, 1, 1]",0.0038,8,7


_________________________________________________
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.8 с
СКОп = 1.0, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 25%, Рэф_мин = 0
Вариант 1: Рэф = 52%, Рэф_мин = 0
Вариант 2: Рэф = 53%, Рэф_мин = 0
Вариант 3: Рэф = 74%, Рэф_мин = 0
Ка = 0.99
Nf = 9, Nf_max = 11
Iter = 8, Iter_max = 10
t = 0.0028, t_max = 0.0077


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t,Nf,Iter
0,0.0,30.000,98.786,11.58,359.8,31.195,96.352,12.57,30.54,29.331,30.78,31.323,1.00,"[0, 1, 1, 1]",0.0044,10,9
1,0.0,30.000,75.064,11.64,0.0,28.304,75.736,10.62,25.34,34.283,25.14,31.843,0.99,"[0, 1, 1, 1]",0.0077,10,9
2,0.0,30.000,78.479,3.31,0.1,31.146,109.232,3.30,10.19,28.629,10.36,27.677,1.05,"[0, 0, 0, 0]",0.0036,8,7
3,0.0,30.000,99.144,11.50,0.2,28.205,103.381,10.28,30.42,29.213,30.31,26.332,1.04,"[0, 0, 0, 1]",0.0044,10,9
4,0.0,30.000,174.065,9.57,0.0,30.208,175.369,10.44,8.40,16.138,8.23,15.265,0.94,"[0, 1, 1, 1]",0.0025,8,7


_________________________________________________


In [6]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2)
    algorithm_name = 'Метод N пеленгов'
    dict_results = tma.swarm(n=100, algorithm_name=algorithm_name, target_func=target_func, p0=p0, seeded=True, verbose=True)
    df = tests.get_df(dict_results).dropna(axis=1)
    tma.print_verbose()

    print('Показатели Монте-Карло:')
    for i in range(4):
        a = df['Точ'].apply(lambda x: x[i])
        print('Вариант {}: Рэф = {:.0f}%, Рэф_мин = {:.0f}'.format(i, 100 * a.mean(), min(a)))
    print('Ка = {:.2f}'.format(df['Ка'].mean()))
    print('t = {:.4f}, t_max = {:.4f}'.format(df['t'].mean(), max(df['t'])))
    
    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['Кб'], axis=1).head())

    tests.save_df(df, name='../tests/test1/d30/N, ' + str(np.degrees(tma.standart_deviation)) + '.xlsx')
    print('_________________________________________________')

Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.6 с
СКОп = 0.0, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.00
t = 0.0004, t_max = 0.0021


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t
0,0.0,30.000,98.786,11.58,0.0,30.000,98.786,11.58,30.5,29.331,30.545,29.33,0.00,"[1, 1, 1, 1]",0.001
1,0.0,30.000,75.064,11.64,0.0,30.000,75.064,11.64,25.3,34.283,25.337,34.28,0.00,"[1, 1, 1, 1]",0.001
2,0.0,30.000,78.479,3.31,0.0,30.000,78.479,3.31,10.2,28.629,10.188,28.63,0.00,"[1, 1, 1, 1]",0.001
3,0.0,30.000,99.144,11.50,0.0,30.000,99.144,11.50,30.4,29.213,30.422,29.21,0.00,"[1, 1, 1, 1]",0.002
4,0.0,30.000,174.065,9.57,0.0,30.000,174.065,9.57,8.4,16.138,8.400,16.14,0.00,"[1, 1, 1, 1]",0.000


_________________________________________________
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.5 с
СКОп = 0.1, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 63%, Рэф_мин = 0
Вариант 1: Рэф = 78%, Рэф_мин = 0
Вариант 2: Рэф = 78%, Рэф_мин = 0
Вариант 3: Рэф = 92%, Рэф_мин = 0
Ка = 0.01
t = 0.0003, t_max = 0.0007


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t
0,0.0,30.000,98.786,11.58,359.8,30.424,96.469,12.06,30.5,29.331,30.565,30.40,0.01,"[1, 1, 1, 1]",0.000
1,0.0,30.000,75.064,11.64,359.8,29.952,72.993,12.07,25.3,34.283,25.312,34.92,0.01,"[1, 1, 1, 1]",0.000
2,0.0,30.000,78.479,3.31,0.0,29.867,86.626,3.12,10.2,28.629,10.171,27.91,0.01,"[0, 1, 1, 1]",0.000
3,0.0,30.000,99.144,11.50,359.8,30.274,96.748,11.89,30.4,29.213,30.411,30.12,0.01,"[1, 1, 1, 1]",0.001
4,0.0,30.000,174.065,9.57,360.0,29.867,174.240,9.59,8.4,16.138,8.360,15.97,0.01,"[1, 1, 1, 1]",0.000


_________________________________________________
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.5 с
СКОп = 0.2, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 44%, Рэф_мин = 0
Вариант 1: Рэф = 58%, Рэф_мин = 0
Вариант 2: Рэф = 59%, Рэф_мин = 0
Вариант 3: Рэф = 66%, Рэф_мин = 0
Ка = 0.05
t = 0.0003, t_max = 0.0007


...

_________________________________________________
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.5 с
СКОп = 0.5, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 0%, Рэф_мин = 0
Вариант 1: Рэф = 0%, Рэф_мин = 0
Вариант 2: Рэф = 0%, Рэф_мин = 0
Вариант 3: Рэф = 1%, Рэф_мин = 0
Ка = 0.37
t = 0.0003, t_max = 0.0009


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t
0,0.0,30.000,98.786,11.58,359.1,23.082,89.166,9.20,30.5,29.331,29.897,23.79,0.46,"[0, 0, 0, 0]",0.000
1,0.0,30.000,75.064,11.64,359.2,21.831,65.545,9.13,25.3,34.283,24.524,26.10,0.46,"[0, 0, 0, 0]",0.001
2,0.0,30.000,78.479,3.31,0.2,25.351,143.400,2.91,10.2,28.629,9.525,20.21,0.34,"[0, 0, 0, 0]",0.000
3,0.0,30.000,99.144,11.50,359.1,21.960,89.070,8.51,30.4,29.213,29.517,22.42,0.55,"[0, 0, 0, 0]",0.001
4,0.0,30.000,174.065,9.57,360.0,26.877,176.906,9.28,8.4,16.138,7.625,13.26,0.28,"[0, 0, 0, 0]",0.001


_________________________________________________
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.5 с
СКОп = 1.0, tau = 2
Показатели Монте-Карло:
Вариант 0: Рэф = 0%, Рэф_мин = 0
Вариант 1: Рэф = 0%, Рэф_мин = 0
Вариант 2: Рэф = 0%, Рэф_мин = 0
Вариант 3: Рэф = 0%, Рэф_мин = 0
Ка = 3.22
t = 0.0004, t_max = 0.0007


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Точ,t
0,0.0,30.000,98.786,11.58,358.2,10.559,78.196,2.90,30.5,29.331,26.151,9.63,5.49,"[0, 0, 0, 0]",0.001
1,0.0,30.000,75.064,11.64,358.4,10.849,51.253,3.68,25.3,34.283,21.400,11.81,4.37,"[0, 0, 0, 0]",0.000
2,0.0,30.000,78.479,3.31,0.4,18.087,181.406,4.69,10.2,28.629,6.744,9.91,2.46,"[0, 0, 0, 0]",0.000
3,0.0,30.000,99.144,11.50,358.2,9.940,75.841,2.53,30.4,29.213,25.255,8.91,6.42,"[0, 0, 0, 0]",0.000
4,0.0,30.000,174.065,9.57,359.9,20.829,182.539,8.35,8.4,16.138,4.934,8.23,1.86,"[0, 0, 0, 0]",0.000


_________________________________________________
